In [1]:
import sys, cv2
import numpy as np
import tracktor_revised as tr
import matplotlib.pyplot as plt
import pickle
import os
import multiprocessing
import logging

In [2]:
# Open video file.
video_file        = '../Data/SF_n2_t2_3113.mp4'
cap               = cv2.VideoCapture(video_file)
n_frames          = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps               = cap.get(cv2.CAP_PROP_FPS)
print(f'{n_frames} frames, {fps} frames per second.')

# Contour detection parameters.
block_size,offset = 15,13
min_area,max_area = 20,1000
n_pix             = 5

vid_info = cap,n_frames,fps,block_size,offset,min_area,max_area,n_pix

56214 frames, 30.0 frames per second.


In [3]:
import multiprocessing
multiprocessing.cpu_count()

4

In [ ]:
# List of frames to analyze.
frameID_list = list(range(5000,50000,5000))
#frameID_list = range(5000,n_frames,1)

def fishCreator(i,vid_info=vid_info):
    print(i)
    sys.stdout.flush()
    
    cap,n_frames,fps,block_size,offset,min_area,max_area,n_pix = vid_info
    
    # Load the frame.
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    ret,frame = cap.read()

    # Detect the contours.
    contours  = tr.contour_detect(frame,min_area,max_area,block_size,offset,n_pix)

    # Make a mask with value j inside fish number j and -1 elsewhere. 
    mask      = 0*cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) - 1
    for j in range(len(contours)):
        cv2.drawContours(mask, contours, j, j, -1)

    # Make and save a list of each fish's pixels.
    pixels=[]
    subFish=[]
    for j in range(len(contours)):
        pixels=(np.nonzero(mask==j))
        n =len(pixels[0]) # Number of pixels in J
        subSubFish=[]
        for k in range(n):
            row=pixels[0][k]
            col=pixels[1][k]
            rgb=frame[row,col]
            grey=round(.3*rgb[0]+.59*rgb[1]+.11*rgb[2],2)
            subSubFish.append([row,col,grey]) #Pixel
        subFish.append(np.array(subSubFish)) #Fish
    return subFish

p = multiprocessing.Pool(processes=2)
fish = p.map(fishCreator,frameID_list)
    
#with open('fishTest.pkl','wb') as fh:
     #pickle.dump(fish,fh)

In [15]:
print(fish)

NameError: name 'fish' is not defined